### Automation Tool to download File links from a given website

In [14]:
! pip install selenium beautifulsoup4 >/dev/null 2>&1 #old trick not to printout the output
! pip install --upgrade selenium >/dev/null 2>&1
! pip install --upgrade webdriver-manager >/dev/null 2>&1

The system cannot find the path specified.


  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/91/08/10cff8463b3510b78f9e3dcef6b37c542b06d71ed1240a8940ba0c75d3bc/selenium-4.26.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions~=4.9 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client~=1.8 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16141199bf9acb7537a3/websocket_client-1.8.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.4/9.7 MB 8.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/9.7 MB 10.2 MB/s eta 0:00:01
   ------ --------------------------------- 1.7/9.7 MB 11.8 MB/s eta 0:00:01
   ----------- ---------------------------- 2.7/9.7 


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b5/b5/3bd0b038d80950ec13e6a2c8d03ed8354867dc60064b172f2f4ffac8afbe/webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata
  Attempting uninstall: webdriver-manager
    Found existing installation: webdriver-manager 3.8.6
    Uninstalling webdriver-manager-3.8.6:
      Successfully uninstalled webdriver-manager-3.8.6



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import time
import requests
import platform
from time import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor, as_completed
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import selenium
selenium.__version__

'4.26.1'

#### Enum Objects


In [3]:
from enum import Enum

class WebDriverPaths(Enum):
    WINDOWS = "../Drivers/Windows/chromedriver.exe"
    MAC = "E:\oneDrive\OneDrive - UCB-O365\classes\data-mining\datamining-project\firstDraft\data-preprocessing\cleaning\Drivers\Mac\chromedriver"

#### Set environmental variables

In [5]:
website_url = "https://www.sec.gov/data-research/financial-statement-notes-data-sets"
driver_env = WebDriverPaths.WINDOWS.value #Default
download_directory = 'firstDraft/data-preprocessing/cleaning/Data/NVDA/unprocessedData' 
chunk_size = 10 * 1024 * 1024
headers = {'User-Agent': 'vijaysbOT/1.'}

if platform.system() == "Darwin":
    driver_env = WebDriverPaths.MAC.value
elif platform.system() == "Windows":
    print("This is Windows")
    driver_env = WebDriverPaths.WINDOWS.value

    


This is Windows


In [6]:
! dpkg -L chromium-chromedriver

'dpkg' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")

In [ ]:
# def download_sec_files(download_directory,  max_files=50):
#     if not os.path.exists(download_directory):
#         os.makedirs(download_directory)
#     chrome_options = webdriver.ChromeOptions()
#     prefs = {'download.default_directory': download_directory}
#     chrome_options.add_experimental_option('prefs', prefs)
#     # chrome_options.add_argument("--headless")  # Run in headless mode
#     # chrome_options.add_argument("--disable-gpu")
#     # chrome_options.add_argument("--no-sandbox")
#     # chrome_options.add_argument("--disable-dev-shm-usage")
#     headers = {'User-Agent': 'vijaysbOT/1.'} # Bypass the bot detection
    
#     try:
#         driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
#     except Exception as e:
#         print(f"Error initializing WebDriver: {e}")
#         return

#     try:
#         driver.get(website_url)

#         download_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.zip')]")

#         for index, link in enumerate(download_links[:max_files]):
#             file_url = link.get_attribute('href')
#             if file_url.endswith('.zip'):

#                 file_name = os.path.basename(file_url)
#                 file_path = os.path.join(download_directory, file_name)

#                 response = requests.get(file_url, headers=headers, stream = True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(chunk_size=chunk_size):
#                             file.write(chunk)
#                     print(f"Downloaded ({index + 1}/{max_files}): {file_name}")
#                 else:
#                     print(f"Failed to download ({index + 1}/{max_files}): {file_name}")


#                 time.sleep(3)
#     finally:
#         driver.quit()

# download_sec_files(download_directory, max_files=1)

Downloaded (1/1): 2024_10_notes.zip


In [8]:
def download_file(file_url, index, max_files):
    file_name = os.path.basename(file_url)
    file_path = os.path.join(download_directory, file_name)
    try:
        response = requests.get(file_url, headers=headers, stream=True, timeout=200)
        response.raise_for_status()
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=chunk_size):
                file.write(chunk)
        print(f"Downloaded ({index + 1}/{max_files}): {file_name}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download ({index + 1}/{max_files}): {file_name} - {e}")

In [ ]:
def download_sec_files(download_directory,  max_files=50):
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)
    chrome_options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': download_directory}
    chrome_options.add_experimental_option('prefs', prefs) 
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    except Exception as e:
        print(f"Error initializing WebDriver: {e}")
        return
    try:
        driver.get(website_url)
        download_links = driver.find_elements(By.XPATH, "//a[contains(@href, '.zip')]")
        download_links = download_links[:max_files]  # Limit to max_files
        file_urls = [link.get_attribute('href') for link in download_links]
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = {executor.submit(download_file, url, idx, max_files): idx for idx, url in enumerate(file_urls)}
            for future in as_completed(futures):
                future.result()
    finally:
        driver.quit()

download_sec_files(download_directory, max_files=55)

Downloaded (5/55): 2024_06_notes.zip
Downloaded (2/55): 2024_09_notes.zip
Downloaded (4/55): 2024_07_notes.zip
Downloaded (8/55): 2024_03_notes.zip
Downloaded (1/55): 2024_10_notes.zip
Downloaded (10/55): 2024_01_notes.zip
Downloaded (3/55): 2024_08_notes.zip
Downloaded (11/55): 2023_12_notes.zip
Downloaded (13/55): 2023_10_notes.zip
Downloaded (7/55): 2024_04_notes.zip
Downloaded (6/55): 2024_05_notes.zip
Downloaded (12/55): 2023_11_notes.zip
Downloaded (15/55): 2023q2_notes.zip
Downloaded (9/55): 2024_02_notes.zip
Downloaded (17/55): 2022q4_notes.zip
Downloaded (18/55): 2022q3_notes.zip
Downloaded (14/55): 2023q3_notes.zip
Downloaded (19/55): 2022q2_notes.zip
Downloaded (16/55): 2023q1_notes.zip
Downloaded (20/55): 2022q1_notes.zip
Downloaded (23/55): 2021q2_notes.zip
Downloaded (22/55): 2021q3_notes.zip
Downloaded (21/55): 2021q4_notes.zip
Downloaded (27/55): 2020q2_notes.zip
Downloaded (25/55): 2020q4_notes.zip
Downloaded (24/55): 2021q1_notes.zip
Downloaded (26/55): 2020q3_notes.z